In [1]:
'''Plan:
1. Load libraries, load data.
2. Preliminary EDA, dealing with missing values, merging train and test.
3. EDA, deleting variables.
4. Feature engineering, ohc.
5. Sample formation.
6. Feature scaling.
7. Model fitting.
8. Performance evaluation.
9. Predictions.
'''

# aside:
# when coding for interview ML purposes or Kaggle, never drop any obervations!
# you will have to make predictions for all obs in test sample.

# correct way to deal with missing obs and merge train and test samples:
# 1. Load both samples.
# 2. Impute missing values in both samples, using train sample to impute missing values.
# 3. Concatentate them into df.


### 1. Load libraries ###

import numpy as np
import pandas as pd
import os, warnings, random, time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer

from xgboost import XGBClassifier

warnings.filterwarnings("ignore")
#os.getcwd()

In [2]:
### Load data ###

train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
print(train.shape)

test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')

train = train.sample(n=50000)

(957919, 120)


In [3]:
### 2. ###

train.shape
train.describe()

# are there numerical features?

un_colval = pd.DataFrame([[x,len(train[x].unique())] for x in train.columns], columns = ['colname', 'n_unique'])
un_colval.loc[un_colval.n_unique < 100]
# all columns contain numerical features

,colname,n_unique
119,claim,2


In [11]:
#train0 = train.copy()
#train = train0.copy()

In [4]:
# given huge difference in accuracy with and without missing values, i suggest creating dummies for them.
# to write more general code, will have to create union of mis_cols for train and test.

train_mis_cols = [col for col in train.columns if train[col].isnull().any()]
test_mis_cols = [col for col in test.columns if test[col].isnull().any()]

for col in train_mis_cols:
    train[col + '_miss'] = (train[col].isnull()).astype(int)

for col in test_mis_cols:
    test[col + '_miss'] = (test[col].isnull()).astype(int)

In [15]:
train

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f109_miss,f110_miss,f111_miss,f112_miss,f113_miss,f114_miss,f115_miss,f116_miss,f117_miss,f118_miss
682454,682454,0.098222,0.49814,1962.400,0.039480,0.50761,-1.48620,336.430,49739.0,4.866100e+15,...,0,0,0,0,0,0,0,0,0,0
775441,775441,0.100890,0.43360,4898.800,0.006724,0.31073,-0.65103,3795.500,795450.0,1.455800e+14,...,0,0,0,0,0,0,0,0,0,0
349573,349573,0.123720,0.48111,1584.700,0.006001,0.17354,1.69800,1762.200,213400.0,7.848400e+15,...,0,0,0,0,0,0,0,0,0,0
577077,577077,0.100010,0.47666,902.080,0.052631,0.34145,0.99562,2371.900,186950.0,3.232100e+15,...,0,0,0,0,0,0,0,0,0,0
835145,835145,0.099639,NaN,1632.300,0.403470,0.46886,0.32978,2000.700,274220.0,2.103800e+15,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815374,815374,0.055024,0.48326,7928.500,0.046148,0.44481,-3.53350,-22.124,321040.0,6.053300e+15,...,0,0,0,0,0,1,0,0,0,0
102276,102276,0.101200,0.48012,244.490,0.112510,0.34945,-2.98280,2505.600,418060.0,8.072800e+13,...,0,0,0,0,0,0,0,0,0,0
94927,94927,0.128640,0.35725,59.309,0.149790,0.27945,0.47255,2655.200,37296.0,-4.003500e+12,...,0,0,0,0,0,0,0,0,0,0
799164,799164,0.156920,0.48848,10637.000,0.020328,0.21993,-1.63840,131.860,212000.0,1.815800e+15,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#train.describe()
#test.describe()

# imputing missing values #

colmnames = train.columns
test['claim'] = np.nan
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(train)
train = pd.DataFrame(imp.transform(train))
test = pd.DataFrame(imp.transform(test))
#train.describe()

In [6]:
train['sample'] = 'train'
test['sample'] = 'pred'
df = pd.concat([train, test])
#df.colnames = list(colmnames)+'sample'
df

,0,1,2,3,4,5,6,7,8,9,...,229,230,231,232,233,234,235,236,237,sample
0,729538.0,0.131690,0.444760,2617.70,0.004542,0.28646,-0.566790,40.83,46886.0,-6.484500e+12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
1,575499.0,0.130780,0.419920,5683.20,0.227520,0.46524,0.491870,341.60,587020.0,1.747700e+15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
2,662663.0,0.101560,0.002789,2393.60,0.301370,0.18251,-1.746000,1439.40,418770.0,6.674500e+14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
3,95680.0,0.030495,0.514900,6877.50,0.110840,0.32847,-1.876900,1784.40,162930.0,3.070700e+14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
4,454978.0,0.058061,0.485070,11964.00,0.330550,0.38319,-1.407400,2073.80,56285.0,7.408900e+15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493469,1451388.0,-0.009112,0.308190,637.64,0.778200,0.41415,-1.068500,651.22,985000.0,6.079700e+15,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,pred
493470,1451389.0,0.088922,0.482650,6924.10,0.025963,0.35540,-0.870200,2514.20,18004.0,6.073500e+14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pred
493471,1451390.0,0.140620,0.484750,1797.10,0.147020,0.28803,-1.407100,434.03,333050.0,2.351000e+15,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pred
493472,1451391.0,0.168000,0.351760,454.79,0.164580,0.16983,0.323850,2331.20,223980.0,-2.795300e+12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pred


In [7]:
newcolnames = list(colmnames) + ['sample']
df.columns = newcolnames
df

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110_miss,f111_miss,f112_miss,f113_miss,f114_miss,f115_miss,f116_miss,f117_miss,f118_miss,sample
0,729538.0,0.131690,0.444760,2617.70,0.004542,0.28646,-0.566790,40.83,46886.0,-6.484500e+12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
1,575499.0,0.130780,0.419920,5683.20,0.227520,0.46524,0.491870,341.60,587020.0,1.747700e+15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
2,662663.0,0.101560,0.002789,2393.60,0.301370,0.18251,-1.746000,1439.40,418770.0,6.674500e+14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
3,95680.0,0.030495,0.514900,6877.50,0.110840,0.32847,-1.876900,1784.40,162930.0,3.070700e+14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
4,454978.0,0.058061,0.485070,11964.00,0.330550,0.38319,-1.407400,2073.80,56285.0,7.408900e+15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493469,1451388.0,-0.009112,0.308190,637.64,0.778200,0.41415,-1.068500,651.22,985000.0,6.079700e+15,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,pred
493470,1451389.0,0.088922,0.482650,6924.10,0.025963,0.35540,-0.870200,2514.20,18004.0,6.073500e+14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pred
493471,1451390.0,0.140620,0.484750,1797.10,0.147020,0.28803,-1.407100,434.03,333050.0,2.351000e+15,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pred
493472,1451391.0,0.168000,0.351760,454.79,0.164580,0.16983,0.323850,2331.20,223980.0,-2.795300e+12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pred


In [8]:
### 3. ###

# we have many variables, are all of them useful? lets see correlation

#cors = df.corr()
#cors.loc[((cors > 0.8)&(cors<1)).any(1)]
# there are no pairwise correlations above 80%

In [9]:
### 4. ###

# check skew and possibly transform some variables #


In [10]:
### 5. ###

y_train = df.loc[df['sample']=='train',['claim']]
X_train = df.drop(columns=['claim'])
X_train = X_train.loc[X_train['sample']=='train']
X_pred = X_train.loc[X_train['sample']=='test']

print(y_train.shape, X_train.shape)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=2)

print(y_train.shape, X_train.shape, X_test.shape)

(50000, 1) (50000, 238)
(45000, 1) (45000, 238) (5000, 238)


In [11]:
X_train.drop(columns = ['sample'], inplace=True)
X_test.drop(columns = ['sample'], inplace=True)
X_pred.drop(columns = ['sample'], inplace=True)

In [24]:
X_train.columns[0:119]

Index(['id', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9',
       ...
       'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117',
       'f118'],
      dtype='object', length=119)

In [12]:
### 6. Scaling ###

ss = StandardScaler()

for i in X_train.columns[0:119]:
    X_train[[i]] = ss.fit_transform(X_train[[i]])
    X_test[[i]] = ss.transform(X_test[[i]])

X_train.describe()  

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f109_miss,f110_miss,f111_miss,f112_miss,f113_miss,f114_miss,f115_miss,f116_miss,f117_miss,f118_miss
count,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,4.500000e+04,...,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000
mean,8.706122e-17,3.066165e-16,8.147557e-17,-9.146264e-17,-1.037343e-16,3.078525e-16,2.129161e-18,9.505483e-17,-1.262743e-16,-4.480367e-17,...,0.016378,0.014933,0.015622,0.016867,0.017600,0.016511,0.017044,0.016622,0.016933,0.016800
std,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,...,0.126925,0.121287,0.124010,0.128773,0.131494,0.127432,0.129438,0.127853,0.129023,0.128523
min,-1.734529e+00,-5.421682e+00,-2.518096e+00,-2.078263e+00,-1.331297e+00,-2.168099e+00,-5.595988e+00,-1.442557e+00,-1.172805e+00,-1.225903e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-8.699589e-01,-4.577459e-01,-4.157796e-01,-5.673929e-01,-7.726334e-01,-4.358995e-01,-4.952981e-01,-8.850075e-01,-8.243322e-01,-7.650833e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.465769e-03,-7.762291e-03,2.892447e-01,-4.307851e-01,-3.034138e-01,1.526149e-01,-1.505378e-01,-1.442736e-01,-2.559475e-01,-5.505530e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.597882e-01,5.904224e-01,7.613421e-01,6.351786e-02,4.398127e-01,7.379532e-01,4.646879e-01,6.780546e-01,5.211848e-01,5.540387e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.731024e+00,7.152474e+00,1.173556e+00,5.511195e+00,5.057952e+00,1.665928e+00,5.224056e+00,2.975679e+00,4.284726e+00,3.637734e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
### 7. Model fitting ###

time1 = time.time()

log_rg = LogisticRegression()

grid_values = {'penalty': ['l2'], 'C': [0.01, 0.1, 0.3, 1, 3, 10]}

lr = GridSearchCV(log_rg, param_grid = grid_values, cv=4)
lr.fit(X_train, y_train)

print('logistic', lr.best_score_, lr.best_params_, time.time()-time1)
# after dropping missing I had 86%. while imputing them with median, i had 52%. with miss dummies, 73%.

logistic 0.7273777777777778 {'C': 1, 'penalty': 'l2'} 14.74954080581665


In [ ]:
time1 = time.time()

svm = svm.SVC(kernel='rbf')

grid_values = {'C':[0.01, 0.1, 1, 5, 10]}

svm = GridSearchCV(svm, param_grid = grid_values, cv=2)
svm.fit(X_train, y_train)

print('SVM', svm.best_score_, svm.best_params_, time.time()-time1)

In [18]:
time1 = time.time()

xgbcl = XGBClassifier(tree_method='gpu_hist', gpu_id=0 )

grid_values = {'n_estimators':[100,200,300],'eta':[0.07, 0.1, 0.13], 'max_depth':[3,4,5]}

xgb = GridSearchCV(xgbcl, param_grid = grid_values, cv=2)
xgb.fit(X_train, y_train)

print('XGBoost', xgb.best_score_, xgb.best_params_, time.time()-time1)

[20:54:13] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:54:14] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:54:14] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:54:15] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be